In [1]:
import numpy as np
np.random.seed(1337)
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline


from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque
import random
import urllib, json
import re
from datetime import datetime
import requests


Using TensorFlow backend.


In [2]:
memory = deque(maxlen=2000)
gamma = 0.95  
epsilon = 0.01  # exploration rate
epsilon_min = 0.01
epsilon_decay = 0.99
learning_rate = 0.001
batch_size = 32
memory = []
action_hist = []
class Account:
    cash = 100000
    amount_held = 0
    def calc_value(self,current_price):
        return self.amount_held * current_price
account = Account()

In [3]:
def build_network(input_sz):
    model = Sequential()
    model.add(Dense(125,input_dim=input_sz, activation='relu'))
    model.add(Dense(50,activation='relu'))
    model.add(Dense(2, activation='linear'))
#     try:
#         model.load_weights("model.h5")
#         print "LOADEd weights"
#     except:
#         pass
    model.compile(loss='mse',optimizer=Adam(lr=learning_rate)) 
    return model

In [4]:
def get_data(symbol):
    url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&outputsize=full&symbol="+symbol+"&apikey=TOGGSCK5JLEEJTGL"
    url_rsi = "https://www.alphavantage.co/query?function=RSI&symbol="+symbol+"&interval=daily&time_period=10&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    url_sma = "https://www.alphavantage.co/query?function=SMA&symbol="+symbol+"&interval=daily&time_period=10&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    url_cci = "https://www.alphavantage.co/query?function=CCI&symbol="+symbol+"&interval=daily&time_period=10&series_type=close&apikey=TOGGSCK5JLEEJTGL"
    
    response_prices = requests.get(url)
    response_rsi = requests.get(url_rsi)
    response_sma = requests.get(url_sma)
    response_cci = requests.get(url_cci)
    
    data = response_prices.json()
    data_rsi = response_rsi.json()
    data_sma = response_sma.json()
    data_cci = response_cci.json()
    
    
    data = data["Time Series (Daily)"]
#     print(data_rsi)
    data_rsi = data_rsi["Technical Analysis: RSI"]
    data_sma = data_sma["Technical Analysis: SMA"]
    data_cci = data_cci["Technical Analysis: CCI"]
    
    ds = {}
    ds["date"] = []
    ds["close"] = []
    ds["open"] = []
    ds["volume"] = []
    ds["high"] = []
    ds["low"] = []
    ds["rsi"] = []
    ds["sma"] = []
    ds["cci"] = []
    
    i = 0
    for time_point in data:
        try:
            ds["date"] += [time_point]
            ds["close"] += [float(data[time_point]["4. close"])]
            ds["open"] += [float(data[time_point]["1. open"])]
            ds["volume"] += [float(data[time_point]["5. volume"])]
            ds["high"] += [float(data[time_point]["2. high"])]
            ds["low"] += [float(data[time_point][ "3. low"])]
        except KeyError as e:
            continue
        try:
            ds["rsi"] += [float(data_rsi[time_point]["RSI"])]
        except KeyError as e:
            ds["rsi"] += [sum(ds["rsi"])/len(ds["rsi"])]
        try:
            ds["sma"] += [float(data_sma[time_point]["SMA"])]
        except KeyError as e:
            ds["sma"] += [sum(ds["sma"])/len(ds["sma"])]
        try:
            ds["cci"] += [float(data_cci[time_point]["CCI"])]
        except KeyError as e:
            ds["cci"] += [sum(ds["cci"])/len(ds["cci"])]
    
    df=pd.DataFrame(ds)
    return df
        


In [5]:
df = get_data("TSLA")

In [6]:
df = df.sort_values(by='date')

In [7]:
def remember(s,a,r,sp):
    memory.append((s,a,r,sp))

In [8]:
def update(memory):
#     Take random sample from memory
    batch = random.sample(memory, batch_size) 
#     Train network on batch
    for s, a, r, sp in batch:
#       if state is final there is no future rewards
        target = r
#       discounted future rewards
    
        target = r + gamma * \
           np.amax(model.predict(sp)[0])
                
#       Get pred then replace action with our target
        target_f = model.predict(s)
        target_f[0][a] = target
        model.fit(s, target_f, epochs=1, verbose=0)

In [9]:
def get_act(obs):
#     if np.random.rand() <= epsilon:
#         # The agent acts randomly
#         return env.action_space.sample()
  
    # Predict the reward value based on the given state
    act_values = model.predict(obs)

    # Pick the action based on the predicted reward
    return np.argmax(act_values[0])

In [10]:
df.iloc[7].close

17.399999999999999

In [11]:
def act(act,day):
#     if np.random.rand() <= epsilon:
#         # The agent acts randomly
# #         epsilon = epsilon * epsilon_decay
#         act = random.uniform(0, 1)
    if act:
        if (account.cash - day[1]["close"]) >= 0:
            account.amount_held =  account.amount_held + 1
            account.cash = account.cash - day[1]["close"]
            current_val = account.calc_value(day[1]["close"])
            i = day[0]
            future = df.iloc[i+7]
            if (account.calc_value(future.close) - current_val) > 0:
                return 1
            else:
                return -1
    #         Buy some shares reward is if we make money in a week
    else:
        if account.amount_held > 0:
            account.amount_held = account.amount_held -1
            account.cash = account.cash + (1*day[1]["close"])
        current_val = account.calc_value(day[1]["close"])
        i = day[0]
        future = df.iloc[i+7]
        if (account.calc_value(future.close) - current_val) < 0:
            return 1
        else:
            return -1
    
    return 0
#         Sell some shares reward is if we avoid losing money in a week

In [12]:
del df["date"]
# for d in df.iterrows():
#     print d[1]["date"]
    

In [13]:
df

cci   close      high       low      open        rsi         sma  \
1455   16.066695   23.89   25.0000   17.5400   19.0000  53.516676  146.679907   
1362   17.221320   23.83   30.4192   23.3000   25.7900  53.738251  146.791034   
851    10.403227   21.96   25.9200   20.2700   25.0000  52.712541  152.448028   
1110   14.485071   19.20   23.1000   18.7100   23.0000  53.413735  146.602501   
849    10.403227   16.11   20.0000   15.8300   20.0000  52.712541  152.448028   
850    10.403227   15.80   16.6300   14.9800   16.4000  52.712541  152.448028   
104    -7.530905   17.46   17.5200   15.5700   16.1400  48.565501  171.545143   
106    -8.366469   17.40   17.9000   16.5500   17.5800  48.634390  173.457302   
1059   11.932612   17.05   18.0700   17.0000   17.9500  53.194000  146.926507   
1058  -35.801500   18.14   18.6400   16.9000   17.3938  53.194000   19.084000   
1062    5.134000   19.84   20.1500   17.7601   17.9400  34.362900   18.679000   
1061   64.471300   19.89   21.5000   19.0000   19.9400  34.643300   18.285000   
1060  109.566900   20.64   21.3000   20.0500   20.7000  38.987100   18.153000   
1057  135.567400   21.91   22.2500   20.9200   21.3700  45.768700   18.424000   
580    75.907100   20.30   21.8500   20.0500   21.8500  39.573000   18.843000   
581    44.814100   20.22   20.9000   19.5000   20.6600  39.279500   19.285000   
578    63.420400   21.00   21.2500   20.3700   20.5000  43.796300   19.639000   
579    77.988600   21.29   21.5600   21.0600   21.1900  45.471900   20.028000   
582    46.029800   20.95   21.5000   20.3000   21.5000  43.771900   20.418000   
583     2.844500   20.55   21.1800   20.2600   20.9100  41.732400   20.659000   
584   -16.230200   20.72   20.9000   20.5100   20.5500  42.986900   20.747000   
585   -89.484500   20.35   20.8800   20.0000   20.7700  40.859600   20.793000   
707  -143.601200   19.94   20.4400   19.5500   20.2000  38.513000   20.723000   
1116   21.353000   20.92   20.9700   20.3330   20.5000  46.650200   20.624000   
1115  161.949000   21.95   21.9500   20.8200   21.0000  53.791500   20.789000   
1113  105.726000   21.26   22.1800   20.8500   21.9500  48.917600   20.893000   
1112  -28.209800   20.45   21.5500   20.0500   21.5400  43.747300   20.838000   
1307 -159.651500   19.59   20.1601   19.5200   20.1000  38.897400   20.668000   
1117 -121.540100   19.60   19.9800   19.4500   19.9000  38.984800   20.533000   
956  -135.021600   19.03   19.6500   18.8200   19.6500  35.746700   20.381000   
...          ...     ...       ...       ...       ...        ...         ...   
1475   37.772100  335.07  339.6000  332.5100  336.8900  49.000900  332.689000   
1393  -50.088600  323.47  341.4900  321.0400  335.5000  40.517200  333.079000   
574  -136.467100  319.57  324.4500  316.1300  323.0000  38.055800  332.212000   
573  -106.417500  325.89  327.1200  311.2200  318.9400  44.163300  332.275000   
1544  117.763800  347.09  350.0000  343.1500  345.3300  59.168200  333.992000   
577   138.788000  356.91  357.2700  343.3000  347.0000  64.129400  336.843000   
576   124.943200  355.17  359.4800  352.7500  357.3500  62.631200  338.108000   
1550  130.808100  365.22  368.5800  357.4000  357.5300  67.503400  340.670000   
1551  102.475200  363.53  370.0000  358.9500  361.0000  65.898100  342.638000   
1418   64.344100  355.40  366.6500  354.6600  361.6000  58.466000  344.732000   
1689   49.938300  357.87  361.2600  353.6200  356.9700  59.989300  347.012000   
1694   74.216100  363.80  367.6600  362.6000  364.6300  63.554800  351.045000   
1693   57.979900  362.33  365.4900  359.3700  365.2000  62.032200  355.321000   
1696   69.281200  362.91  366.5000  362.5200  363.0000  62.426800  359.023000   
1420  -74.848000  351.92  363.3000  351.5900  361.2100  51.218900  359.506000   
1697 -166.859900  347.46  354.0000  345.8000  352.9100  47.382900  358.561000   
40   -210.328400  337.86  345.8200  331.8500  345.8200  40.185100  356.830000   
41   -131.109500  341.35  342.2400  337.3700

In [16]:
model = build_network(len(df.columns))
prv_obs = None
i = 0
gl = []
for day in df.iterrows():
    i = 1 + i
    if i == len(df) - 10:
        break
    obs = day[1]
    obs_values = np.reshape(obs.values,[1,len(df.columns)])
    action = get_act(obs_values )
    if action == 1:
        action_hist += ["buy"] 
    else:
        action_hist += ["sell"]
    if i % 100 == 0:
        if action == 1:
            print "Buying" 
        else:
            print "Selling"
        print "day" , i
        print "cash",account.cash
        print "shares held",account.amount_held
        print "net worth",account.calc_value(day[1]["close"]) + account.cash

#     print(action)
    reward = act(action,day)
#     print reward
    gl += [(account.calc_value(day[1]["close"]) + account.cash) - 100000]
    if prv_obs is not None and reward is not None:
        remember(prv_obs,action,reward,obs_values)
    prv_obs = obs_values  
#     if done:
#         model.save_weights("model.h5")
#         print("Episode finished after {} timesteps".format(t+1))
#         break
    if len(memory) >= batch_size:
        update(memory)

Buying
day 100
cash 99777.85
shares held 9
net worth 100043.26
Buying
day 200
cash 99311.476
shares held 25
net worth 99927.726
Buying
day 300
cash 99059.378
shares held 35
net worth 99866.828
Selling
day 400
cash 99014.558
shares held 37
net worth 100099.768
Buying
day 500
cash 98395.888
shares held 55
net worth 100253.788
Selling
day 600
cash 98829.7971
shares held 41
net worth 100125.9834
Buying
day 700
cash 98397.2913
shares held 51
net worth 100532.1513
Buying
day 800
cash 88025.4398
shares held 151
net worth 113544.4398
Buying
day 900
cash 72190.0269
shares held 251
net worth 116014.6269
Buying
day 1000
cash 51243.2309
shares held 351
net worth 130962.3509
Buying
day 1100
cash 26836.0347
shares held 451
net worth 135166.2347
Buying
day 1200
cash 5563.8547
shares held 551
net worth 117471.9547
Buying
day 1300
cash 105.2797
shares held 576
net worth 129613.1197
Buying
day 1400
cash 105.2797
shares held 576
net worth 114556.4797
Buying
day 1500
cash 105.2797
shares held 576
net wort

In [18]:
print ((sum(gl)/len(gl))/100000)*100

print (194712.6397/100000)*100
# print action_hist

19.2457321142
194.7126397
